## Import SCAPE and other packages and connect to database.  

NOTE: sometimes you have to type your password in the notebook server terminal, not here in the notebook (a python2/python3 difference).

In [1]:
import scape.registry as R
import scape.sql as sql
import os,json
from getpass import getuser,getpass

import sqlalchemy
from sqlalchemy import Column, Integer, String, Numeric, DATETIME
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

In [2]:
#Prepare the URL with authentication data

pwd = getpass('Postgres password: ')
myurl = sqlalchemy.engine.url.URL(
    'postgresql',         # SQL backend type
    username='postgres',  # or get_user() if username is the same
    password = pwd,       #  
    host='localhost',     # assuming you're on a workstation
    database='lanldb',    # Name of your LANL database
    port=5432)            # Make sure this matches your database configuration

Postgres password: ········


In [3]:
# Create a persistent engine
global engine
engine=sqlalchemy.create_engine( myurl )

In [4]:
def connect(engine):
    """Read the registry file and connect the metadata to the database object"""
    with open('lanldataregistry.json','rt') as f:
        overlay = json.load(f)
    
    tmd={k:sql.SqlDataSource(engine=engine,
                             metadata=R.TableMetadata(v),
                             table=k,
                             description='') for k,v in overlay.items()}
    reg = R.Registry(tmd)
    return reg

In [5]:
registry=connect(engine)

## Examples of of using registry to explore and understand data 

In [6]:
#Get one of the data streams
dns=registry['dns']

In [7]:
#inspect the knowledge engineering on it
dns

Field,Dim,Tags
id,primarykey,
resolvedhost,hostname,"dst,dns,dnsresponse"
shost,hostname,src
time,time,seconds


In [8]:
# Select all the data and have a look at it in a pandas dataframe
q1=dns.select().pandas()
q1

,time,shost,resolvedhost
0,2,C4653,C5030
1,2,C5782,C16712
2,6,C1191,C419
3,15,C3380,C22841
4,18,C2436,C5030
5,31,C161,C2109
6,35,C5642,C528
7,38,C3380,C22841
8,42,C2428,C1065
9,42,C2428,C2109


In [9]:
# Ask SCAPE to return to you only fields that have dimension hosthame
q2=dns.select("hostname").run()
q2

,resolvedhost,shost
0,C5030,C4653
1,C16712,C5782
2,C419,C1191
3,C22841,C3380
4,C5030,C2436
5,C2109,C161
6,C528,C5642
7,C22841,C3380
8,C1065,C2428
9,C2109,C2428


In [10]:
# Get some quick statistics to find out which host asks a lot of DNS questions in an hour
q1.groupby(["shost"]).size().sort_values(ascending=False)

shost
C743      375434
C585      295478
C1823      95021
C3380      91745
C395       81332
C5741      75489
C2492      53472
C4858      41379
C17490     37572
C62        35697
C23542     35138
C561       33952
C22235     31776
C8129      28057
C639       23723
C8882      22863
C2013      20188
C17640     19163
C17925     16532
C5778      16009
C15059     15965
C24165     14206
C2897      12430
C3392      12384
C1621      11439
C768       11368
C20084     11172
C6810      10981
C1763      10802
C5786      10638
           ...  
C22055         1
C18215         1
C25466         1
C23578         1
C22638         1
C26483         1
C15778         1
C25545         1
C8182          1
C22531         1
C22102         1
C21738         1
C23916         1
C1584          1
C24077         1
C23901         1
C1660          1
C4662          1
C2042          1
C23687         1
C21144         1
C22529         1
C23373         1
C17722         1
C18025         1
C20418         1
C23413         1
C20413  

In [11]:
# Ask SCAPE to filter to a particular host, figure out what hosts the talkative host is resolving
q3=dns.select().where("hostname=='C3380'").run()
q3

,time,shost,resolvedhost
0,15,C3380,C22841
1,38,C3380,C22841
2,76,C3380,C22841
3,100,C3380,C22841
4,137,C3380,C22841
5,162,C3380,C22841
6,198,C3380,C22841
7,224,C3380,C22841
8,259,C3380,C22841
9,286,C3380,C22841


In [12]:
# Wow, lots of the resolutions go to a particular outside host.  Let's count up
q3.groupby(["resolvedhost"]).size().sort_values(ascending=False)

resolvedhost
C22841    84856
C2013      1523
C798        735
C4222       553
C5136       542
C5640       525
C22529      514
C5588       434
C3380       418
C25295      325
C5728       325
C23521      315
C1685       245
C1707       236
C25294      181
C22139      169
C22167       93
C1200        29
C2012        22
C612         17
C528         14
C529         14
C625         14
C586         14
C1065        10
C457         10
C17337        8
C2109         6
C467          6
C18085        4
C3349         4
C4827         2
dtype: int64

In [13]:
#Who else is resolving that host?
q4=dns.select().where("hostname=='C22841'").pandas()
q4

,time,shost,resolvedhost
0,15,C3380,C22841
1,38,C3380,C22841
2,76,C3380,C22841
3,100,C3380,C22841
4,137,C3380,C22841
5,162,C3380,C22841
6,198,C3380,C22841
7,224,C3380,C22841
8,259,C3380,C22841
9,286,C3380,C22841


In [14]:
q4.groupby(["shost"]).size().sort_values(ascending=False)
#so I guess nobody else asked for it.

shost
C3380    84856
C2013    18608
C976         1
dtype: int64

In [15]:
# Pivot to flows and see if that host is talking to any other internal hosts
flows=registry['flows']

In [16]:
flows.select().where("hostname =='C3380'").run()

,time,duration,shost,sport,dhost,dport,protocol,pktcount,bytecount


In [17]:
flows.select().where("hostname =='C22841'").run()

,time,duration,shost,sport,dhost,dport,protocol,pktcount,bytecount


In [18]:
## Hmm, neither one talked at all...  Let's see if there are interesting flows in this
## Actually, no, I'm going to check it in and go to bed...